## IMPORTS

In [1]:
import pandas as pd
import inflection
import math
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.display import HTML       

### Helper functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()



In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


### Loading data

In [4]:
df_sales_raw=pd.read_csv('Datasets/train.csv', low_memory=False)
df_store_raw=pd.read_csv('Datasets/store.csv', low_memory=False)

# merge

df_raw=pd.merge(df_sales_raw,df_store_raw, how='left', on='Store')

In [5]:
df_raw.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## DATA DESCRIPTION

In [9]:
df1=df_raw.copy()
df1.columns


Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

### Rename columns

In [ ]:
cols_old=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase=lambda x: inflection.underscore(x)

cols_new=list(map(snakecase,cols_old))

#rename 
df1.columns=cols_new

### Data dimensions

In [ ]:
print('Numbers of raw:{}'.format(df1.shape[0]))
print('Numbers of cols:{}'.format(df1.shape[1]))

### Data Type

In [ ]:
df1['date']=pd.to_datetime(df1['date'])

df1.dtypes


### Check NA

In [ ]:
df1.isna().sum()

### Fillout NA

In [ ]:
df1['competition_distance'].max()

In [ ]:
#competition_distance              
df1['competition_distance']=df1['competition_distance'].apply(lambda x:200000.0 if math.isnan(x) else x)

#competition_open_since_month  
df1['competition_open_since_month']=df1.apply(lambda x: x['date'].month 
                                              if math.isnan(x['competition_open_since_month']) 
                                              else x['competition_open_since_month'], axis=1)

#competition_open_since_year  
df1['competition_open_since_year']=df1.apply(lambda x: x['date'].year 
                                              if math.isnan(x['competition_open_since_year']) 
                                              else x['competition_open_since_year'], axis=1)

#promo2_since_week  
df1['promo2_since_week']=df1.apply(lambda x: x['date'].week 
                                              if math.isnan(x['promo2_since_week']) 
                                              else x['promo2_since_week'], axis=1)
#promo2_since_year 
df1['promo2_since_year']=df1.apply(lambda x: x['date'].year 
                                              if math.isnan(x['promo2_since_year' ]) 
                                              else x['promo2_since_year'], axis=1)
#promo_interval   
month_map={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
df1['promo_interval'].fillna(0,inplace=True)
df1['month_map']=df1['date'].dt.month.map(month_map)
df1['is_promo']=df1[['promo_interval','month_map']].apply(
    lambda x:0 if x['promo_interval']==0 else 1 if x['month_map'] in x['promo_interval'].split(',')else 0, axis=1)

In [ ]:
df1.isna().sum()

### Change Types

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month']=df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year']=df1['competition_open_since_year'].astype(int)
df1['promo2_since_week']=df1['promo2_since_week'].astype(int)
df1['promo2_since_year']=df1['promo2_since_year'].astype(int)

### Descriptive Statistical

In [ ]:
num_atributes=df1.select_dtypes(include=['int64','float64'])
cat_atributes=df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])


#### numerical Attributes

In [ ]:
# central Tendency - mean, median
ct1=pd.DataFrame(num_atributes.apply(np.mean)).T
ct2=pd.DataFrame(num_atributes.apply(np.median)).T


In [ ]:
# Dispersion - std, min, max, range, skew, kurtosis
d1=pd.DataFrame(num_atributes.apply(np.std)).T
d2=pd.DataFrame(num_atributes.apply(np.min)).T
d3=pd.DataFrame(num_atributes.apply(np.max)).T
d4=pd.DataFrame(num_atributes.apply(lambda x: x.max()-x.min())).T
d5=pd.DataFrame(num_atributes.apply(lambda x: x.skew())).T
d6=pd.DataFrame(num_atributes.apply(lambda x: x.kurtosis())).T

 # concatenate
m=pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns=['attributes','min','max','range','mean','median','std','skew','kurtosis']
m

In [ ]:
sns.set_style('white')
sns.displot(df1['sales'], bins=40);
